# Retail Demo Store - Personalization Workshop - Lab 1

Welcome to the Retail Demo Store Personalization Workshop. In this module we're going to be implementing multiple personalization use cases, powered by [Amazon Personalize](https://aws.amazon.com/personalize/), to enhance the base Retail Demo Store storefront. We will break up the process into multiple labs that are designed to progressively build on top of each other. Therefore, you should complete the labs in the order specified below. Some labs are optional, as indicated below.

## Workshop overview

- **Lab 1**: Introduction and dataset preparation (this lab) (_25 minutes_)
- **Lab 2**: Prepare Amazon Personalize and import datasets (_20 minutes_)
- **Lab 3**: Create e-commerce domain recommenders and custom solutions in Amazon Personalize (_60 minutes_)
- **Lab 4**: Evaluate recommendations from Amazon Personalize recommenders and campaigns, activate real-time events using an Event Tracker, and apply business rules using filters (_30-40 minutes_)
- Real-time events with CDPs
    - **Lab 5**: Segment real-time data (optional) (_45 minutes_)
    - **Lab 6**: mParticle (optional) (_45 minutes_)
- **Lab 7**: Intelligent user segmentation using Amazon Personalize (_20-30 minutes_)
- **Lab 8**: Personalized offers (optional) (_120 minutes_)
- **Lab 9**: Cleanup resources (_15 minutes_)

## Overview of use cases

Through the labs in the workshop we will be adding personalization to several views in the Retail Demo Store.

- **Navigation**:
    - Personalized search: the order of search results displayed in the auto-complete search dropdown are personalized to each user's interests.
- **Home page**:
    - Recommended For You: the "Inspired by your shopping trends" grid is populated with personalized recommendations for the current user (if emulating a shopper or after a few interactions have been collected for a cold user).
    - Featured products personalized ranking: order featured products based on the current user's interests.
- **Product detail**:
    - Related product recommendations: the "Compare similar items" carousel is populated with related item recommendations based on co-interactions of other items and thematic similarities between items.
- **Category detail**:
    - Personalized ranking: the order of items displayed on the category detail pages are personalized to the current user's interests.
- **Interactive video stream**:
    - Related product recommendations: the "Compare similar items" carousel is populated with related item recommendations based on co-interactions of other items and thematic similarities between items.

In addition, the recommenders we build in this workshop will be foundational components of other Retail Demo Store workshops including the customer messaging workshop, conversational AI workshop, and location services workshop.

## Introduction to Amazon Personalize

Amazon Personalize makes it easy for customers to develop applications with a wide array of personalization use cases, including real time product recommendations and customized direct marketing. Amazon Personalize brings the same machine learning technology used by Amazon.com to everyone for use in their applications – with no machine learning experience required. Amazon Personalize customers pay for what they use, with no minimum fees or upfront commitment. You can start using Amazon Personalize with a simple three step process, which only takes a few clicks in the AWS console, or a set of simple API calls. First, point Amazon Personalize to user data, catalog data, and activity stream of views, clicks, purchases, etc. in Amazon S3 or upload using a simple API call. Second, with a single click in the console or an API call, train a custom private recommendation model for your data. Third, retrieve personalized recommendations for any user by creating a recommender or campaign, and using the GetRecommendations API.

In this lab we are going to focus on inspecting and preparing the data we will need to provide to Personalize.

## Setup

The workshop will be using the python programming language and the AWS SDK for python. Even if you are not fluent in python, the code cells should be reasonably intuitive. In practice, you can use any programming language supported by the AWS SDK to complete the same steps from this workshop in your application environment.

### Update dependencies

To get started, we need to perform a bit of setup. First, we need to ensure that a current version of botocore is locally installed. The botocore library is used by boto3, the AWS SDK library for python. We need a current version to be able to access some of the newer Amazon Personalize features.

The following cell will update pip and install the latest botocore library.

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore

### Import dependencies

Next we need to import some dependencies/libraries needed to complete this lab.

In [ ]:
import boto3
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import requests
import csv  
import os

%matplotlib inline

### Create cients

Next we need to create the AWS service clients needed for this workshop.

- **personalize**: this client is used to create resources in Amazon Personalize
- **servicediscovery**: this client is used to lookup the local IP addresses of the Retail Demo Store microservices that we'll need in the workshop
- **ssm**: this client is used to access application configuration details stored in the Systems Manager parameter store

Finally we'll lookup an identifier, stored as a resource tag in the SageMaker instance at deployment time. We need this tag to lookup resources we need, such as the IAM role needed by Personalize

In [ ]:
# Setup clients
personalize = boto3.client('personalize')
servicediscovery = boto3.client('servicediscovery')
ssm = boto3.client('ssm')

# The Uid is a unique ID and we need it to find the role made by CloudFormation
with open('/opt/ml/metadata/resource-metadata.json') as f:
    data = json.load(f)
sagemaker = boto3.client('sagemaker')
sagemakerResponce = sagemaker.list_tags(ResourceArn=data["ResourceArn"])
for tag in sagemakerResponce["Tags"]:
    if tag['Key'] == 'Uid':
        Uid = tag['Value']
        break

print('Uid:', Uid)

### Lookup S3 bucket and initialize file names

We will be configuring some variables that will store the location of our source data. When the Retail Demo Store stack was deployed in this account, an Amazon S3 bucket was created for you and the name of this bucket was stored in Systems Manager Parameter Store. Using the `ssm` boto3 client we created above, we can retrieve the name of the bucket.

In [ ]:
bucketresponse = ssm.get_parameter(
    Name='retaildemostore-stack-bucket'
)

# We will use this bucket to store our training data:
bucket = bucketresponse['Parameter']['Value']

# We will build and upload our training data in these files:
items_filename = "items.csv"
users_filename = "users.csv"
interactions_filename = "interactions.csv"

print('Bucket: {}'.format(bucket))

## Fetch, Prepare, and Upload User, Product, and Interaction Data

Amazon Personalize provides predefined recipes, based on common use cases, for training models. A recipe is a machine learning algorithm that you use with settings, or hyperparameters, and the data you provide to train an Amazon Personalize model. The data you provide to train a model are organized into separate datasets by the type of data being provided. A collection of datasets are organized into a dataset group. The three dataset types supported by Personalize are items, users, and interactions. Depending on the recipe type you choose, a different combination of dataset types are required. For all recipe types, an interactions dataset is required. Interactions represent how users interact with items. For example, viewing a product, watching a video, listening to a recording, or reading an article. For this workshop, we will be using a recipes that supports all three dataset types.

When we deployed the Retail Demo Store, it was deployed with an initial seed of fictitious User and Product data. We will use this data to train multiple models (recommenders and solutions) in the Amazon Personalize service which will be used to serve product recommendations, related items, and to rerank product lists for our users. The User and Product data can be accessed from the Retail Demo Store's [Users](https://github.com/aws-samples/retail-demo-store/tree/master/src/users) and [Products](https://github.com/aws-samples/retail-demo-store/tree/master/src/products) microservices, respectively. We will access our data through microservice data APIs, process the data, and upload them as CSVs to S3. Once our datasets are in S3, we can import them into the Amazon Personalize service.

Let's get started.

### Get Products Service Instance

We will be pulling our Product data from the [Products Service](https://github.com/aws-samples/retail-demo-store/tree/master/src/products) that was deployed in Amazon Elastic Container Service as part of the Retail Demo Store. To connect to this service we will use [AWS Cloud Map](https://aws.amazon.com/cloud-map/)'s Service Discovery to discover an instance of the Product Service running in ECS, and then connect directly to that service instances to access our data.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='products',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

assert len(response['Instances']) > 0, 'Products service instance not found; check ECS to ensure it launched cleanly'

products_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Products Service Instance IP: {}'.format(products_service_instance))

#### Download and Explore the Products Dataset

Next, let's call the Products service to fetch all products in the catalog and then load that product information into a [Pandas dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). Once in a dataframe, we can display, inspect, and manipulate data as needed. A snapshot of the dataframe is displayed to give you an idea of the fields available in the catalog for each product.

In [ ]:
response = requests.get('http://{}/products/all'.format(products_service_instance))
products = response.json()
products_df = pd.DataFrame(products)
pd.set_option('display.max_rows', 5)

products_df

#### Prepare items dataset

When training models in Amazon Personalize, we can provide structured and unstructured metadata about our items. This data helps improve the relevance of recommendations and is particularly useful when recommending new/cold items added to your catalog. 

For this workshop we will be using retail domain recommenders for some of our personalization use cases as well as creating some custom solutions for other use cases. The retail domain recommenders were released at re:Invent 2021. You can read more about them in the [Personalize blog](https://aws.amazon.com/blogs/machine-learning/amazon-personalize-announces-recommenders-optimized-for-retail-and-media-entertainment/).

The retail domain recommenders stipulate some [reserved fields/columns](https://docs.aws.amazon.com/personalize/latest/dg/ECOMMERCE-datasets-and-schemas.html) that we must conform to. For this workshop we will use the `price`, `category`, `style`, `description`, and `gender_affinity` fields from the Retail Demo Store catalog add map them to the `PRICE`, `CATEGORY_L1`, `CATEGORY_L2`, `PRODUCT_DESCRIPTION`, and `GENDER` columns expected by Personalize. Personalize will automatically apply a natural language processing (NLP) machine learning model to the product description column to extract features from the text. The product's unique identifier is required. 

The following cell creates a new dataframe populated with just the columns we intend to use for the items dataset and then renames the columns.

In [ ]:
products_dataset_df = products_df[['id','price','category','style','description','gender_affinity']]
products_dataset_df = products_dataset_df.rename(columns = {'id':'ITEM_ID',
                                                            'price':'PRICE',
                                                            'category':'CATEGORY_L1',
                                                            'style':'CATEGORY_L2',
                                                            'description':'PRODUCT_DESCRIPTION',
                                                            'gender_affinity':'GENDER'})

### Inspect items dataset

Let's first get a sense for the dimensions of the item catalog dataframe. The following cell displays the number of rows and columns in the dataframe.

In [ ]:
products_dataset_df.shape

Now let's make sure that we don't have any missing/null values for each column. Otherwise, Personalize will return an error for columns/fields that it expects to be fully populated. Note that Personalize supports specifying fields that are not fully populated but for the Retail domain recommender fields in the items dataset, all fields for all rows must be fully populated. The following cell will display the list the count of non-null values for each field.

In [ ]:
products_dataset_df.info()

Notice that the `GENDER` column has a null value for many rows. This is because the source `gender_affinity` field in the product catalog is used to indicate men's and women's products in the clothing, accessories, and beauty categories and is null for products from other categories such as furniture and electronics. So we must replace all null values for this column with a suitable value indicating that the product is appropriate for any gender. The following cell we will replace all null column values with `Any`.

In [ ]:
products_dataset_df['GENDER'].fillna('Any', inplace = True)

In [ ]:
products_dataset_df.info()

In [ ]:
products_dataset_df

### Save to CSV and upload to S3 bucket

Our items dataset is now ready. The following cell will save the dataframe to a local CSV file and then upload the CSV to the staging S3 bucket.

In [ ]:
products_dataset_df.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(items_filename).upload_file(items_filename)

### Get Users Service Instance

Next, let's prepare the users dataset with information on the users pre-loaded into the Retail Detail Store.

We will be pulling our User data from the [Users Service](https://github.com/aws-samples/retail-demo-store/tree/master/src/users) that is deployed as part of the Retail Demo Store. To connect to this service we will use Service Discovery to discover an instance of the User Service, and then connect directly to that service instance to access our data.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='users',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

assert len(response['Instances']) > 0, 'Users service instance not found; check ECS to ensure it launched cleanly'

users_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Users Service Instance IP: {}'.format(users_service_instance))

#### Download and Explore the Users Dataset

Similar to the Products microservice above, we'll query the Users microservice to retrieve information on all pre-loaded users. Along the way we'll remove all users without a gender or persona (i.e. users added to the storefront after deployment).

In [ ]:
response = requests.get('http://{}/users/all?count=10000'.format(users_service_instance))
users = response.json()
users_df = pd.DataFrame(users)
# Remove any users without a persona or gender (i.e. possibly created in web UI)
users_df = users_df[(users_df['persona'].str.strip().astype(bool)) | (users_df['gender'].str.strip().astype(bool))]
pd.set_option('display.max_rows', 5)

users_df

#### Prepare and Upload Data

Similar to the items dataset we created above, we can provide metadata on our users when training models in Personalize. For this workshop we will include each user's age and gender. Unlike the columns in the items dataset, there are no [column naming requirements for the users dataset](https://docs.aws.amazon.com/personalize/latest/dg/ECOMMERCE-users-dataset.html). As before, we will use uppercase column names to match the Personalize convention, save the data as a CSV, and upload to our S3 bucket.

In [ ]:
users_dataset_df = users_df[['id','age','gender']]
users_dataset_df = users_dataset_df.rename(columns = {'id':'USER_ID','age':'AGE','gender':'GENDER'}) 

users_dataset_df.to_csv(users_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(users_filename).upload_file(users_filename)

### Create User-Items Interactions Dataset

To mimic user behavior, we will be generating a new dataset that represents user interactions with items.
To make the interactions more realistic, we will use the pre-defined shopper persona for each user to generate event
types for products matching that persona. We will create events for viewing products, adding products to a cart,
checking out, and completing orders. 
The effect of discounts on interactions depends on a separate "discount persona" stored against the user.
The script also makes an effort to keep interactions balanced between and within categories and products.

For more information about how this script works, take a look at the script file at [generate_interactions_personalize.py](../../generators/generate_interactions_personalize.py).

We'll start by preparing the script and inputs to be called from this notebook. Don't worry about understanding the nuances of the generator script configuration. In a production workload you'd be using actual user interactions from your clickstream analytics and order managemement systems rather than generating them as we do below.

In [ ]:
sys.path.insert(0, os.path.abspath('../../generators'))

import generate_interactions_personalize as gi

In [ ]:
# Where to put the generated data
gi.GENERATED_DATA_ROOT = "./"

# Let us keep things deterministic so we can replicate the data.
gi.RANDOM_SEED = 0

# Interactions will be generated between these dates 
# (we keep things deterministic by keeping fixed dates)
gi.FIRST_TIMESTAMP = 1591803782  # 2020-06-10, 18:43:02
gi.LAST_TIMESTAMP = 1599579782  # 2020-09-08, 18:43:02

# Minimum number of interactions to generate
gi.min_interactions = 675000

# Users are set up with 3 product categories on their personas, 
# such as beauty_electronics_outdoors
# [. If [0.6, 0.25, 0.15] it means 60% of the time they'll want to
# choose a product from the first category (beauty in this example),
# 25% from 2nd, etc.
gi.CATEGORY_AFFINITY_PROBS = [0.6, 0.25, 0.15]

# With this probability a product interaction will be with the product discounted
# Here we go the other way - what is the probability that a product that a user is already interacting
# with is discounted - (depending on whether user likes discounts or not,
# a feature available on their simulated profile as discount_persona)
gi.DISCOUNT_PROBABILITY = 0.2
gi.DISCOUNT_PROBABILITY_WITH_PREFERENCE = 0.5

# After interacting with a product, there are this many products within 
# the category that a user is likely to jump on next.
# The purpose of this constant is to keep recommendations focused
# if there are too many products in a category. 
gi.PRODUCT_AFFINITY_N = 4

# from 0 to 1. If 0 then products in busy categories get represented less. If 1 then all products same amount.
gi.NORMALISE_PER_PRODUCT_WEIGHT = 1.0

# Show progress every 30 seconds. The script takes some time to complete.
gi.PROGRESS_MONITOR_SECONDS_UPDATE = 30

# Percentages of each event type to generate
gi.product_added_percent = .08
gi.cart_viewed_percent = .05
gi.checkout_started_percent = .02
gi.order_completed_percent = .01

The following cell actually runs the interaction generator script. This will take a few minutes to complete. Progress will be displayed as it runs.

In [ ]:
%%time

gi.generate_interactions(interactions_filename, users_df, products_df)

#### Open and Explore the Simulated Interactions Dataset

Let's take a closer look at the generated interactions file. First let us see a few lines of the raw CSV data:

In [ ]:
!head -n 5 $interactions_filename

Now let us load it as a Pandas dataframe. Note:

- An EVENT_TYPE column which can be used to train different Personalize campaigns and also to filter on recommendations.
- The custom DISCOUNT column which is a contextual metadata field, that Personalize reranking campaign and user personalization recommenders can take into account to predict on the best next product based the user's propensity to interact with discount products.

In [ ]:
interactions_df = pd.read_csv(interactions_filename)
interactions_df

Chart the counts of each `EVENT_TYPE` generated for the interactions dataset. We're simulating a site where visitors heavily view/browse products and to a lesser degree add products to their cart and checkout.

In [ ]:
categorical_attributes = interactions_df.select_dtypes(include = ['object'])

plt.figure(figsize=(16,3))
chart = sns.countplot(data = categorical_attributes, x = 'EVENT_TYPE')
plt.xticks(rotation=90, horizontalalignment='right')
plt.show()

Let us plot product persona vs product category distribution - so that you can see what categories get assigned to which types of persona. Note that because of attempts to reach balance between categories and products in the generation script, the proportions do not exactly match those configured.

In [ ]:
merged_df = interactions_df[['USER_ID', 'ITEM_ID']].astype({'USER_ID':str, 'ITEM_ID':str})
merged_df = merged_df.merge(users_df[['id', 'persona']], left_on='USER_ID', right_on='id').drop(columns=['id', 'USER_ID'])
merged_df = merged_df.merge(products_df[['id', 'category']], left_on='ITEM_ID', right_on='id').drop(columns=['id', 'ITEM_ID'])
merged_df
plot_df = merged_df.groupby(['persona', 'category'])['category'].count().unstack()
sns.heatmap(plot_df, annot=True, fmt="g", cmap='viridis')
plt.title('Heatmap of user persona vs product category')
plt.show()

#### Discount persona vs event type distribution

Let us see how the event distribution came out. We should see a different takeup of discounts between users with different discount personas.

In [ ]:
merged_df = interactions_df.loc[interactions_df.EVENT_TYPE == 'AddToCart'][['USER_ID', 'DISCOUNT']]
merged_df = merged_df[['USER_ID', 'DISCOUNT']].astype({'USER_ID':str}).merge(users_df, left_on='USER_ID', right_on='id')

plot_df = merged_df.groupby(['discount_persona', 'DISCOUNT'])[['id']].count().unstack()
plot_df = plot_df.droplevel(axis='columns', level=0)
plot_df.plot.bar()
plt.title('Event types according to discount persona')
plt.show()

#### Balance over products I

Let us have a careful look at product and category distributions.
The interactions generation script ensures that there are small groups of products users tend to interact with, to maintain
strong training signals. If you look at the script you will see that although we choose
products randomly within a category, they are interacted with in small random groups in the category.


In [ ]:
merged_df = interactions_df[['ITEM_ID', 'USER_ID']].astype({'ITEM_ID': str}).merge(products_df, left_on='ITEM_ID', right_on='id')
plot_df = merged_df.groupby(['USER_ID', 'category']).id.apply(set)
plot_df.apply(len).value_counts().sort_index().plot.bar()
plt.xlabel('Number of different products examined by user.')
plt.title(f'We should have reduced users to a small number of products\n'
          f'maximum size should be {gi.PRODUCT_AFFINITY_N+1}')
# The peak at 1 is the male jewelry - there is only one product
plt.show()

#### Balance over products II
Ensuring that users interact with only overlapping subsets of products is not sufficient. We also ensure there is some balance in interactions between and within
categories explicitly by using balance information when sampling category and to add
interactions depending on the product simulated.

In [ ]:
interactions_df.groupby('ITEM_ID')['USER_ID'].count().plot.bar(figsize=[20,4])
plt.title('Balance over products. \nIf you selected NORMALISE_PER_PRODUCT_WEIGHT=1\nthis should be flatter, otherwise catgories with\n'
          'higher products should be having less per product')
plt.show()

In [ ]:
merged_df = interactions_df[['ITEM_ID', 'USER_ID']].astype({'ITEM_ID': str}).merge(products_df, left_on='ITEM_ID', right_on='id')[['ITEM_ID', 'category', 'USER_ID']]
plot_df = merged_df.groupby(['ITEM_ID', 'category'])['USER_ID'].count()
plot_df.unstack().plot.barh(figsize=[8, 16])
plt.title('Balance over products with category coded.')
plt.show()

#### Prepare and Upload Data

Finally, let's upload the generated interactions CSV to S3 so it can be picked up by Amazon Personalize later in the workshop.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(interactions_filename).upload_file(interactions_filename)

## Lab 1 Summary - What have we accomplished?

In this lab we prepared and inspected the three dataset types (items, users, and interactions) that will be used to train models and create recommenders in Amazon Personalize.

We created a CSV for each dataset type and uploaded them to the Amazon S3 staging bucket created by the Retail Demo Store deployment.

In the next lab we will start creating resources in Amazon Personalize to receive our dataset files.

### Store variables needed in the next lab

We will pass some variables initialized in this lab by storing them in the notebook environment.

In [ ]:
%store Uid
%store bucket
%store items_filename
%store users_filename
%store interactions_filename

print(Uid)
print(bucket)
print(items_filename)
print(users_filename)
print(interactions_filename)

### Continue to Lab 2

Open [Lab 2](./Lab-2-Prepare-Personalize-and-import-data.ipynb) to continue the workshop.